# Load

In [943]:
import pandas as pd
import numpy as np
from collections import Counter
from datetime import datetime
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

In [136]:
# dataset
ss_cols = ['session_id', 'start_timestamp', 'timezone',
       'timezone_offset', 'previous_sessions_duration',
       'user_created_timestamp', 'is_user_first_session', 'is_session',
       'is_developer', 'is_wau', 'is_mau', 'country', 'region', 'city',
       'latitude', 'longitude', 'locale', 'os_name', 'session_index',
       'device_id', 'user_id_hash']

att_value = pd.read_csv('att.csv',usecols=['attribute_value'])
ses = pd.read_csv('sessions.csv',usecols=ss_cols)
eve = pd.read_csv("events.csv")
att = pd.read_csv('att.csv',usecols=['session_id', 'attribute','user_id_hash'])
mes = pd.read_csv('mes.csv')
sub = pd.read_csv('sample_submission_2.csv')
# label
eve['purchase'] = eve[eve['event'] == '8']

In [146]:
print("nrows of att:",len(att_without_hash))
print("nrows of eve:",len(eve))
print("nrows of ses:",len(ses))
print("nrows of mes:",len(mes))
print("ncols of att:",len(att_without_hash.columns))
print("ncols of eve:",len(eve.columns))
print("ncols of ses:",len(ses.columns))
print("ncols of mes:",len(mes.columns))

nrows of att: 185590092
nrows of eve: 111946597
nrows of ses: 6239836
nrows of mes: 2896
ncols of att: 3
ncols of eve: 11
ncols of ses: 21
ncols of mes: 6


# Check NAs

In [242]:
eve.isnull().sum()

session_id         0
event              0
event_value        0
event_month        0
event_day          0
event_dow          0
event_hour         0
purchase           0
event_timestamp    0
event_week         0
user_id_le         0
dtype: int64

In [385]:
att.isnull().sum()

session_id    0
attribute     0
user_id_le    0
dtype: int64

In [394]:
ses.isnull().sum()

session_id                        0
start_timestamp                   0
timezone                       9538
timezone_offset                9538
previous_sessions_duration        0
user_created_timestamp            0
is_user_first_session             0
is_session                        0
is_developer                      0
is_wau                            0
is_mau                            0
country                       16657
region                        12926
city                          12918
latitude                      12918
longitude                     12918
locale                         9538
os_name                       27444
session_index                     0
device_id                     18073
user_id_hash                      0
user_id_le                        0
dtype: int64

In [395]:
mes.isnull().sum()

app_id                   0
message_id               0
action_type              0
delivery_type            0
delivery_time_mode       0
goal_kind             1640
dtype: int64

# Preprocess

In [211]:
# label encode hash, lower size significantly
from sklearn import preprocessing

ur_hash = set(eve['user_id_hash'])|set(att['user_id_hash'])
le = preprocessing.LabelEncoder()
le.fit(np.array(list(ur_hash)))
eve['user_id_le'] = le.transform(eve['user_id_hash'])
att['user_id_le'] = le.transform(att['user_id_hash'])
ses['user_id_le'] = le.transform(ses['user_id_hash'])
sub['user_id_le'] = le.transform(sub['user_id_hash'])
att.drop(['user_id_hash'],axis=1,inplace=True)
eve.drop(['user_id_hash'],axis=1,inplace=True)

In [374]:
att.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 185590092 entries, 0 to 185590091
Data columns (total 3 columns):
session_id    int64
attribute     int64
user_id_le    int64
dtypes: int64(3)
memory usage: 4.1 GB


In [375]:
eve.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 111946597 entries, 15817265 to 108027205
Data columns (total 11 columns):
session_id         int64
event              object
event_value        float64
event_month        int64
event_day          int64
event_dow          int64
event_hour         int64
purchase           int64
event_timestamp    datetime64[ns]
event_week         int64
user_id_le         int64
dtypes: datetime64[ns](1), float64(1), int64(8), object(1)
memory usage: 10.0+ GB


In [578]:
event_cnt = Counter()
for e in eve['event']:
    try: 
        event_cnt[int(e)]+=1
    except:
        continue 

## Preprocess Attribute
Compare those who have been purchased and who haven't

### attribute

In [ ]:
usr_att = att.groupby(['user_id_le'],as_index=False)['attribute']\
.agg({"att_cnt":"count","is67":lambda x:int(67 in x),"is96":lambda x:int(96 in x)})
# purchased
user_pch = pd.DataFrame({"user_id_le":purchases.user_id_le.unique()}).sort_values(by='user_id_le')
user_pch = pd.merge(user_pch,usr_att,on=['user_id_le'],how='left')
user_pch = pd.merge(user_pch,usr_att_val,on=['user_id_le'],how='left')
user_pch = pd.merge(user_pch,usr_att_val2,on=['user_id_le'],how='left')

In [ ]:
# non-purchsed
nonpch = pd.DataFrame({"user_id_le":list(set(eve.user_id_le)-set(user_pch.user_id_le))})
nonpch = pd.merge(nonpch,usr_att,on=['user_id_le'],how='left')
nonpch = pd.merge(nonpch,usr_att_val,on=['user_id_le'],how='left')
nonpch = pd.merge(nonpch,usr_att_val2,on=['user_id_le'],how='left')

### attribute_value

In [ ]:
att_value['length'] = att_value['attribute_value'].apply(lambda x:len(str(x)))
att_value = pd.DataFrame({"user_id_le":att['user_id_le'],"attribute_value":att_value['attribute_value'],'att_val_len':att_value['length']})

usr_att_val = att_value.groupby(['user_id_le'],as_index=False)['attribute_value']\
.agg({"att_val_cnt":"count","att_val_uniq":"nunique"})
usr_att_val2 = att_value.groupby(['user_id_le'],as_index=False)['att_val_len']\
.agg({"att_len_mean":"mean","att_len_max":"max","att_len_min":"min"})

Let's go further on this feature. U can totally skip this chunk since I save all important info in `user_att.csv` and eventually combine those info into training and testing dataset.

Basic idea:
- explore features around **attribute value** on **7 and 14 days**
- create same features for training set with **last** 7 and 14 days, otherwise there would be an infomation leak.

 - **buy_mean** equals to purchasing count in certain period / occuring times in `attribute.csv`. You can imagine the more this value is, the more likely user would go on purchasing recently.
 
 - **bracket**  is tricky feature. During EDA I found some users whose *attribute_value* was encoded with **[]** seldom purchase. Later feature importance of tree models proved this thought.
 
 - **length** is about the length of *attribute_value*, it's useful too due to some values encoded into long and mysterious terms, that might imply sth.

- **att_ses_ct** is a feature extracted from `attribute.csv` merged with `session.csv`. 

## Preprocess Event

In [ ]:
from datetime import datetime

eve['event_timestamp'] = eve['event_timestamp'].apply(lambda x:datetime.fromtimestamp(x/1000))
eve['event_month'] = eve['event_timestamp'].apply(lambda x:x.month)
eve['event_day'] = eve['event_timestamp'].apply(lambda x:x.day)
eve['event_dow'] = eve['event_timestamp'].apply(lambda x:x.dayofweek)
eve['event_hour'] = eve['event_timestamp'].apply(lambda x:x.hour)
# fillna
eve['event_value'] = eve['event_value'].fillna(0)
# delete useless cols
eve.drop(['event_timestamp'],inplace=True,axis=1)
eve.drop(['app_id'],axis=1,inplace=True)
eve = eve.sort_values(by=['user_id_le','event_timestamp'],ascending=False)

## Preprocess Session

In [ ]:
ses['start_timestamp'] = ses['start_timestamp'].apply(lambda x:datetime.fromtimestamp(x/1000))
ses['user_created_timestamp'] = ses['user_created_timestamp'].apply(lambda x:datetime.fromtimestamp(x/1000))
ses['duration_since_start'] = ses['start_timestamp']-ses['user_created_timestamp']
ses_col = [c for c in ses.columns if c not in ['timezone', 'timezone_offset','is_developer','device_id','region', 'city', 'latitude', 'longitude','locale','user_id_hash']]
ses_c = ses[ses_col]

In [ ]:
# tranform time durations into hours
ses['duration_since_start'] = ses['duration_since_start'].apply(lambda x:round(x/(np.timedelta64(1,'h')*24),3))
ses['previous_sessions_duration'] =ses['previous_sessions_duration'].apply(lambda x:round(x/(3600*24),3))

In [ ]:
def mis_impute(data):
    for i in data.columns:
        if data[i].dtype == "object":
            data[i] = data[i].fillna("other")
        elif (data[i].dtype == "int64" or data[i].dtype == "float64"):
            data[i] = data[i].fillna(data[i].mean())
        else:
            pass
    return data
ses = mis_impute(ses)

ses['ses_month'] = ses['start_timestamp'].apply(lambda x:x.month)
# ses['ses_week'] = ses['start_timestamp'].apply(lambda x:x.week)

## Week tags
we need to create week tags manually in this case since a week starts from Saturday to Friday next week here.

In [ ]:
bins_dt = pd.date_range('2018-09-29', freq='7D', periods=12)
bins_str = bins_dt.astype(str).values
labels = [i for i in range(0,len(bins_str)-1)] # 0 = 9.29-10.5,...,9=12.1-12.7,10=12.8-12.14

eve['event_week'] = pd.cut(eve.event_timestamp.astype(np.int64)//10**9,
                   bins=bins_dt.astype(np.int64)//10**9,
                   labels=labels)

ses['ses_week'] = pd.cut(ses.start_timestamp.astype(np.int64)//10**9,
                   bins=bins_dt.astype(np.int64)//10**9,
                   labels=labels)

# Feature Engineering
- Stage1 -FE on `event.csv` and `session.csv`: most of them created around time duration.


- Stage2 - Most features from `attribute.csv`: add our above foundings from attribute data

### Stage1

In [994]:
# function to extract features
def get_time_gap(strs,parm):
    time = strs.split(":")
    time = list(set(time))
    time = sorted(list(map(lambda x:int(x),time)))
    time_gap = []
    # active on that day
    if len(time) == 1:
        return -20

    for index, value in enumerate(time):
        if index <= len(time) - 2:
            gap = abs(time[index] - time[index + 1])
            time_gap.append(gap)

    if parm == '1':
        return np.mean(time_gap)
    elif parm == '2':
        return np.max(time_gap)
    elif parm == '3':
        return np.min(time_gap)
    elif parm == '4':
        return np.std(time_gap)
    elif parm == '5':
        return sp.stats.skew(time_gap)
    elif parm == '6':
        return sp.stats.kurtosis(time_gap)

def get_day_repeat_count(strs):
    time = strs.split(":")
    time = dict(Counter(time))
    time = sorted(time.items(), key=lambda x: x[1], reverse=False)
    # show up once in one day
    if (len(time) == 1) & (time[0][1] == 1):
        return 0
    # show up many times in one day
    elif (len(time) == 1) & (time[0][1] > 1):
        return 1
    # show up many times in multiple days
    elif (len(time) > 1) & (time[0][1] >= 2):
        return 2
    else:
        return 3
    
def get_continue_launch_count(strs,parm):
    time = strs.split(":")
    time = dict(Counter(time))
    time = sorted(time.items(), key=lambda x: x[0], reverse=False)
    key_list = []
    value_list = []
    if len(time) == 1:
        return -2
    for key,value in dict(time).items():
        key_list.append(int(key))
        value_list.append(int(value))

    if np.mean(np.diff(key_list, 1)) == 1:
        if parm == '1':
            return np.mean(value_list)
        elif parm == '2':
            return np.max(value_list)
        elif parm == '3':
            return np.min(value_list)
        elif parm == '4':
            return np.sum(value_list)
        elif parm == '5':
            return np.std(value_list)
    else:
        return -1

def get_contin_day(day_list):
    time = day_list.split(":")
    time = list(map(lambda x:int(x),time))
    m = np.array(time)
    if len(set(m)) == 1:
        return -1
    m = list(set(m))
    if len(m) == 0:
        return -20
    n = np.where(np.diff(m) == 1)[0]
    i = 0
    result = []
    while i < len(n) - 1:
        state = 1
        while n[i + 1] - n[i] == 1:
            state += 1
            i += 1
            if i == len(n) - 1:
                break
        if state == 1:
            i += 1
            result.append(2)
        else:
            i += 1
            result.append(state + 1)
    if len(n) == 1:
        result.append(2)
    if len(result) != 0:
        return np.max(result)

In [ ]:
def Fea_Eng(event,session,tag,add_label=False): 
    
    # event part
    eve_f1 = event.groupby(['user_id_le'],as_index=False)['event_timestamp'].agg({"user_count_{}".format(tag):"count"})
    eve_f2 = event.groupby(['user_id_le'],as_index=False)['event_day'].agg({"user_eve_day_count_{}".format(tag): "nunique"})

    eve_f3 = event[['user_id_le', 'event_day']]
    eve_f3['event_day'] = eve_f3['event_day'].astype('str')

    eve_f3 = eve_f3.groupby(['user_id_le'])['event_day'].agg(lambda x: ':'.join(x)).reset_index()
    eve_f3.rename(columns={'event_day': 'act_list'}, inplace=True)

    # time gap of the event per user
    eve_f3['time_gap_mean_{}'.format(tag)] = eve_f3['act_list'].apply(get_time_gap,args=('1'))
    eve_f3['time_gap_max_{}'.format(tag)] = eve_f3['act_list'].apply(get_time_gap,args=('2'))
    eve_f3['time_gap_min_{}'.format(tag)] = eve_f3['act_list'].apply(get_time_gap,args=('3'))
    eve_f3['time_gap_std_{}'.format(tag)] = eve_f3['act_list'].apply(get_time_gap,args=('4'))

    # active count mean
    eve_f3['mean_act_count_{}'.format(tag)] = eve_f3['act_list'].apply(lambda x: len(x.split(":")) / len(set(x.split(":"))))
    # active date mean
    eve_f3['act_mean_date_{}'.format(tag)] = eve_f3['act_list'].apply(lambda x: np.sum([int(ele) for ele in x.split(":")]) / len(x.split(":")))
    # users show up many times in multiple days?
    eve_f3['day_repeat_count_{}'.format(tag)] = eve_f3['act_list'].apply(get_day_repeat_count)
    # Launch continuously?
    eve_f3['con_act_day_count_mean_{}'.format(tag)] = eve_f3['act_list'].apply(get_continue_launch_count, args=('1'))
    eve_f3['con_act_day_count_max_{}'.format(tag)] = eve_f3['act_list'].apply(get_continue_launch_count, args=('2'))
    eve_f3['con_act_day_count_min_{}'.format(tag)] = eve_f3['act_list'].apply(get_continue_launch_count, args=('3'))
    eve_f3['con_act_day_count_total_{}'.format(tag)] = eve_f3['act_list'].apply(get_continue_launch_count, args=('4'))
    eve_f3['con_act_day_count_std_{}'.format(tag)] = eve_f3['act_list'].apply(get_continue_launch_count, args=('5'))
    eve_f3['con_act_max_{}'.format(tag)] = eve_f3['act_list'].apply(get_contin_day)
    del eve_f3['act_list']

    # when is the next event happen?
    eve_f4 = event[['user_id_le', 'event_timestamp']].sort_values(['user_id_le', 'event_timestamp'],ascending=False)
    eve_f4['next_time'] = eve_f4.groupby(['user_id_le'])['event_timestamp'].diff(1).apply(np.abs)
    eve_f4['next_time'] = eve_f4['next_time'].fillna(-1)
    eve_f4 = eve_f4.groupby(['user_id_le'], as_index=False)['next_time'].agg({
        'next_time_max': np.max,
        'next_time_mean': np.nanmean,
    })
    # convert it into hour
    eve_f4['next_time_max_{}'.format(tag)] = eve_f4['next_time_max'].apply(lambda x:round(x/np.timedelta64(1,'h'),3))
    eve_f4['next_time_mean_{}'.format(tag)] = eve_f4['next_time_mean'].apply(lambda x:round(x/np.timedelta64(1,'h'),3))                                                     
    eve_f4.drop(['next_time_max','next_time_mean'],axis=1,inplace=True)
    
    # purchase cnt
    eve_f5 = event.groupby(['user_id_le'], as_index=False)['purchase'].agg({'user_pch_ct_{}'.format(tag): "sum"})
    
    # session part
    ses_f1 = session.groupby(['user_id_le'],as_index=False)['is_session'].agg({"avg_True_ses_{}".format(tag):"mean"})
    # how many cities correspond to one user id?
    ses_f2 = session.groupby(['user_id_le'],as_index=False)['city'].agg({"city_ct_{}".format(tag):"count"})
    # duration since start
    ses_f3 = session.groupby(['user_id_le'],as_index=False)['duration_since_start'].agg({"du_since_start_{}".format(tag):"max"})
    # duration of previous session
    ses_f4 = session.groupby(['user_id_le'],as_index=False)['previous_sessions_duration'].agg({"prev_ses_du_mean_{}".format(tag):"mean"})
    
    # load target table
    if add_label:
        data = pd.read_csv('targets.csv',usecols=['user_id_hash','user_purchase_binary_7_days','user_purchase_binary_14_days'])
        data['user_id_le'] = le.transform(data['user_id_hash'])
        data = data[[ 'user_id_le', 'user_purchase_binary_7_days','user_purchase_binary_14_days']]
    else:
        data = pd.read_csv('targets.csv',usecols=['user_id_hash'])
        data['user_id_le'] = le.transform(data['user_id_hash'])
        data = data[['user_id_le']]
    # eve
    data = pd.merge(data, eve_f1, on=['user_id_le'], how='left')
    data = pd.merge(data, eve_f2, on=['user_id_le'], how='left')
    data = pd.merge(data, eve_f3, on=['user_id_le'], how='left')
    data = pd.merge(data, eve_f4, on=['user_id_le'], how='left')
    data = pd.merge(data, eve_f5, on=['user_id_le'], how='left')
    # ses
    data = pd.merge(data, ses_f1, on=['user_id_le'], how='left')
    data = pd.merge(data, ses_f2, on=['user_id_le'], how='left')
    data = pd.merge(data, ses_f3, on=['user_id_le'], how='left')
    data = pd.merge(data, ses_f4, on=['user_id_le'], how='left')
    return data

### Stage2

In [1209]:
def add_features(event,data,tag): # event is the recent month, recent two month
    
    # add f1 purchase time gap
    eve_f6 = event[['user_id_le', 'event_timestamp']].sort_values(['user_id_le', 'event_timestamp'],ascending=False)
    eve_f6['next_time'] = event[event['event'] == '8'].groupby(['user_id_le'])['event_timestamp'].diff(1).apply(np.abs)
    eve_f6['next_time'] = eve_f6['next_time'].fillna(-1)
    eve_f6 = eve_f6.groupby(['user_id_le'], as_index=False)['next_time'].agg({
        'next_pur_time_max': np.max,
        'next_pur_time_mean': np.nanmean,
        'next_pur_time_min':np.min
    })
    # convert it into hour
    eve_f6['next_pch_time_max_{}'.format(tag)] = eve_f6['next_pur_time_max'].apply(lambda x:round(x/np.timedelta64(1,'h'),3))
    eve_f6['next_pch_time_mean_{}'.format(tag)] = eve_f6['next_pur_time_mean'].apply(lambda x:round(x/np.timedelta64(1,'h'),3))                                                     
    eve_f6['next_pch_time_min_{}'.format(tag)] = eve_f6['next_pur_time_min'].apply(lambda x:round(x/np.timedelta64(1,'h'),3))                                                     
    eve_f6.drop(['next_pur_time_max','next_pur_time_mean','next_pur_time_min'],axis=1,inplace=True)
    
    # add f2 purchase day gap
    eve_f7 = event[event['event'] == '8'][['user_id_le', 'event_day']]
    eve_f7['event_day'] = eve_f7['event_day'].astype('str')

    eve_f7 = eve_f7.groupby(['user_id_le'])['event_day'].agg(lambda x: ':'.join(x)).reset_index()
    eve_f7.rename(columns={'event_day': 'pch_list'}, inplace=True)

    # time gap of the event per user
    eve_f7['day_pch_gap_mean_{}'.format(tag)] = eve_f7['pch_list'].apply(get_time_gap,args=('1'))
    eve_f7['day_pch_gap_max_{}'.format(tag)] = eve_f7['pch_list'].apply(get_time_gap,args=('2'))
    eve_f7['day_pch_gap_min_{}'.format(tag)] = eve_f7['pch_list'].apply(get_time_gap,args=('3'))
    eve_f7['day_pch_gap_std_{}'.format(tag)] = eve_f7['pch_list'].apply(get_time_gap,args=('4'))

    # pch count mean
    eve_f7['pch_mean_count_{}'.format(tag)] = eve_f7['pch_list'].apply(lambda x: len(x.split(":")) / len(set(x.split(":"))))
    # pch date mean
    eve_f7['pch_mean_date_{}'.format(tag)] = eve_f7['pch_list'].apply(lambda x: np.sum([int(ele) for ele in x.split(":")]) / len(x.split(":")))
    # users show up many times in multiple days?
    eve_f7['pch_day_repeat_count_{}'.format(tag)] = eve_f7['pch_list'].apply(get_day_repeat_count)
    
    # pch continuously?
    eve_f7['con_pch_day_count_mean_{}'.format(tag)] = eve_f7['pch_list'].apply(get_continue_launch_count, args=('1'))
    eve_f7['con_pch_day_count_max_{}'.format(tag)] = eve_f7['pch_list'].apply(get_continue_launch_count, args=('2'))
    eve_f7['con_pch_day_count_min_{}'.format(tag)] = eve_f7['pch_list'].apply(get_continue_launch_count, args=('3'))
    eve_f7['con_pch_day_count_total_{}'.format(tag)] = eve_f7['pch_list'].apply(get_continue_launch_count, args=('4'))
    eve_f7['con_pch_day_count_std_{}'.format(tag)] = eve_f7['pch_list'].apply(get_continue_launch_count, args=('5'))
    eve_f7['con_pch_max_{}'.format(tag)] = eve_f7['pch_list'].apply(get_contin_day)
    del eve_f7['pch_list']
    
    
    data = pd.merge(data,eve_f6,on=['user_id_le'],how='left')
    data = pd.merge(data,eve_f7,on=['user_id_le'],how='left')
    
    return data


In [1338]:
# attribute features merging
def add_att(data):
    data = pd.merge(data,usr_att,on=['user_id_le'],how='left')
    data = pd.merge(data,usr_att_val,on=['user_id_le'],how='left')
    data = pd.merge(data,usr_att_val2,on=['user_id_le'],how='left')
    return data

### Our training plan 
(number here means week tag)


**7 days:**

- $Fea_{0-8}$ + $Fea_{7-8}$ + $Fea_{8}$ $->$ $Purchase_{9}$ (train-optional)
- $Fea_{0-9}$ + $Fea_{8-9}$ + $Fea_{9}$ $->$ $Purchase_{10}$ (train-**used**)
- $Fea_{0-10}$ + $Fea_{9-10}$ + $Fea_{10}$ $->$ $Purchase_{11}$ (predict)

In [1535]:
# test
eve[eve['event_week']==9]['event_timestamp'].max()

Timestamp('2018-12-08 00:00:00.922000')

Specific Dataset used for creating features

In [1019]:
# 7 days for training
eve09 = eve[eve['event_week']<=9]
eve89 = eve[(eve['event_week']<=9)&(eve['event_week']>=8)]
eve9 = eve[eve['event_week']==9]

eve69 = eve[(eve['event_week']<=9)&(eve['event_week']>=6)]
eve710 = eve[(eve['event_week']<=10)&(eve['event_week']>=7)]

ses09 = ses[ses['ses_week']<=9]
ses89 = ses[(ses['ses_week']<=9)&(ses['ses_week']>=8)]
ses9 = ses[ses['ses_week']==9]

df09 = Fea_Eng(eve09,ses09,tag='09',add_label=False)
df89 = Fea_Eng(eve89,ses89,tag='89',add_label=False)
df9 = Fea_Eng(eve9,ses9,tag='9',add_label=True)

df_ = pd.merge(df09, df89, on=['user_id_le'], how='left')
df = pd.merge(df_, df9, on=['user_id_le'], how='left')

In [1056]:
# 7 days for prediction
eve010 = eve[(eve['event_week']<=10)]
eve910 = eve[(eve['event_week']<=10)&(eve['event_week']>=9)]
eve10 = eve[eve['event_week']==10]

ses010 = ses[(eve['event_week']<=10)]
ses910 = ses[(ses['ses_week']<=10)&(ses['ses_week']>=9)]
ses10 = ses[ses['ses_week']==10]

df010 = Fea_Eng(eve010,ses010,tag='010',add_label=False)
df910 = Fea_Eng(eve910,ses910,tag='910',add_label=False)
df10 = Fea_Eng(eve10,ses10,tag='10',add_label=True)

df_p_ = pd.merge(df010, df910, on=['user_id_le'], how='left')
df_p = pd.merge(df_p_, df10, on=['user_id_le'], how='left')

In [1344]:
# Add more features
df = add_features(eve69,df,'rec_mon')
df = add_att(df)

In [ ]:
df_p = add_features(eve710,df_p,'rec_mon')
df_p = add_att(df_p)

**14 days:**

- $Fea_{0-7}$ + $Fea_{6-7}$ + $Fea_{7}$ $->$ $Purchase_{8-9}$ (train-optional)
- $Fea_{0-8}$ + $Fea_{7-8}$ + $Fea_{8}$ $->$ $Purchase_{9-10}$ (train-**used**)
- $Fea_{0-10}$ + $Fea_{9-10}$ + $Fea_{10}$ $->$ $Purchase_{11-12}$ (predict)

In [1057]:
# 14 days for training
eve08 = eve[eve['event_week']<=8]
eve78 = eve[(eve['event_week']<=8)&(eve['event_week']>=7)]
eve8 = eve[eve['event_week']==8]
# add
eve58 = eve[(eve['event_week']<=8)&(eve['event_week']>=5)]

ses08 = ses[eve['event_week']<=8]
ses78 = ses[(ses['ses_week']<=8)&(ses['ses_week']>=7)]
ses8 = ses[ses['ses_week']==8]

df08 = Fea_Eng(eve08,ses08,tag='08',add_label=False)
df78 = Fea_Eng(eve78,ses78,tag='78',add_label=False)
df8 = Fea_Eng(eve8,ses8,tag='8',add_label=True)

df_2w_ = pd.merge(df08, df78, on=['user_id_le'], how='left')
df_2w = pd.merge(df_2w_, df8, on=['user_id_le'], how='left')

In [1349]:
# Add more features
df_2w = add_features(eve58,df_2w,'rec_mon')
df_2w = add_att(df_2w)

Now I don't need to create `df_2w_p` due to it's same as `df_p`, but it would be different eventually.

### Add features about attr_value

In [ ]:
# now we need to split user_att table and apply those in prediction and training data

# to df-->df7
attv_f_tr7 = user_att[['user_id_le','att_ses_ct', '7bracket_tr', '7buy_mean_tr', '7buy_mean_tr>.8_tr',
       '7length<3_tr']]
# to df_2w-->df14
attv_f_tr14 = user_att[['user_id_le','att_ses_ct', '14buy_mean_tr', '14bracket_tr', '14buy_mean_tr>.8_tr',
       '14length<3_tr']]
# to df_p-->df7_p
attv_f_pd7 = user_att[['user_id_le', 'att_ses_ct','7bracket_p', '7buy_mean_p', '7buy_mean>.8_p',
       '7length<3_p']]
# to df_p-->df14_p
attv_f_pd14 = user_att[['user_id_le', 'att_ses_ct','14buy_mean_p', '14bracket_p', '14buy_mean>.8_p',
       '14length<3_p']]

df7 = pd.merge(df,attv_f_tr7,on=['user_id_le'],how='left')
df14 = pd.merge(df_2w,attv_f_tr14,on=['user_id_le'],how='left')
df7_p = pd.merge(df_p,attv_f_pd7,on=['user_id_le'],how='left')
df14_p = pd.merge(df_p,attv_f_pd14,on=['user_id_le'],how='left')

In [1626]:
# many nans we have because time shifting is nearly unlikely to cover all userids.
# put -1 because light gradient boost machine knows -1 is nan
def impute_transfer_save(df,tag):
    for c in df.columns:
        df[c] = df[c].fillna(-1)
    df['user_id_hash'] = le.inverse_transform(df['user_id_le'])
    df.to_csv('{}.csv'.format(tag),index=None)

# Labels

In [1539]:
# filter out only the rows where they've purchased
purchases = eve[eve['event'] == '8']
dec_1 = datetime(2018, 12, 1)
dec_7 = datetime(2018, 12, 8)

seven_day_purchases = set(purchases[purchases['event_timestamp'] > dec_7].user_id_le)
fourteen_day_purchases = set(purchases[purchases['event_timestamp'] > dec_1].user_id_le)

targets = pd.DataFrame({'user_id_le': eve['user_id_le'].unique()})
targets['user_purchase_binary_7_days'] = targets['user_id_le'].apply(
    lambda x: 1 if x in seven_day_purchases else 0)
targets['user_purchase_binary_14_days'] = targets['user_id_le'].apply(
    lambda x: 1 if x in fourteen_day_purchases else 0)

# get previous target for FeaEng
# purchases = eve[eve['event'] == '8']
# dec_1 = datetime(2018, 11, 24)
# dec_7 = datetime(2018, 12, 1)
# dec_15 = datetime(2018, 12, 8)

# seven_day_purchases = set(purchases[(purchases['event_timestamp'] > dec_7)&(purchases['event_timestamp'] < dec_15)].user_id_le)
# fourteen_day_purchases = set(purchases[(purchases['event_timestamp'] > dec_1)&(purchases['event_timestamp'] < dec_15)].user_id_le)

# targets2 = pd.DataFrame({'user_id_le': eve['user_id_le'].unique()})
# targets2['user_purchase_binary_7_days'] = targets2['user_id_le'].apply(
#     lambda x: 1 if x in seven_day_purchases else 0)
# targets2['user_purchase_binary_14_days'] = targets2['user_id_le'].apply(
#     lambda x: 1 if x in fourteen_day_purchases else 0)

In [1540]:
# check
targets2['user_purchase_binary_7_days'].sum()

4729

In [1541]:
targets2['user_purchase_binary_14_days'].sum()

9251

In [1475]:
df.drop(['user_purchase_binary_7_days'],axis=1,inplace=True)
df_2w.drop(['user_purchase_binary_14_days'],axis=1,inplace=True)

In [1476]:
df = pd.merge(df,targets[['user_id_le','user_purchase_binary_7_days']],on=['user_id_le'],how='left')
df_2w = pd.merge(df_2w,targets[['user_id_le','user_purchase_binary_14_days']],on=['user_id_le'],how='left')

In [1477]:
def save(df,tag):
    df.to_csv('{}.csv'.format(tag),index=None)

# Save

In [ ]:
save(df7,'df7_feav3')
save(df14,'df14_feav3')
save(df7_p,'df7_p_feav3')
save(df14_p,'df14_p_feav3')